<a href="https://colab.research.google.com/github/pablogadhi/TranslationDetector/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/pablogadhi/TranslationDetector
!python -m spacy download en
!python -m spacy download es
%cd TranslationDetector
!pip install --upgrade torch torchtext

Cloning into 'TranslationDetector'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 64 (delta 31), reused 44 (delta 18), pack-reused 0
Unpacking objects: 100% (64/64), done.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')
/content/TranslationDetector/TranslationDetector
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already u

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!git pull

Already up to date.


In [0]:
import torch
from translation.train import train_model
from translation.data_loader import load_data, make_iters
from translation.transformer import Transformer

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
!nvidia-smi

Wed Jun 10 19:47:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Load data and create the model

In [0]:
DATA_DIR='/content/drive/My Drive/NLP/data/'

In [15]:
SRC, TGT, train, valid, test = load_data(
        DATA_DIR + "en-es-0_", "es.txt", "en.txt", DATA_DIR + "TGT_Field.pt", DATA_DIR + "SRC_Field.pt")

Loading data...
Data loaded!


In [0]:
train_itr, valid_itr, test_itr = make_iters(
        train, valid, test, device, batch_size=4000)
model = Transformer(len(SRC.vocab), len(TGT.vocab)).to(device)

In [17]:
len(SRC.vocab.itos)

40000

### Train the model

In [18]:
train_model(model, train_itr, valid_itr, SRC, TGT, device, 10, save_at=1, checkpoint_f=DATA_DIR + "checkpoint.pt")

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Step    50 : 397.45 ms/batch | loss  8.05
Step   100 : 346.24 ms/batch | loss  6.98
Step   150 : 342.45 ms/batch | loss  5.86
Step   200 : 347.34 ms/batch | loss  5.46
Step   250 : 350.40 ms/batch | loss  4.78
Step   300 : 342.61 ms/batch | loss  5.15
Step   350 : 349.37 ms/batch | loss  4.62
Step   400 : 342.31 ms/batch | loss  4.85
Step   450 : 343.95 ms/batch | loss  4.37
Step   500 : 344.30 ms/batch | loss  4.88
Step   550 : 342.46 ms/batch | loss  3.84
Step   600 : 343.19 ms/batch | loss  4.56
Step   650 : 345.60 ms/batch | loss  4.36
Step   700 : 340.68 ms/batch | loss  4.46
Step   750 : 342.59 ms/batch | loss  4.46
Step   800 : 346.79 ms/batch | loss  4.16
Step   850 : 346.28 ms/batch | loss  4.44
Step   900 : 339.05 ms/batch | loss  3.91
Step   950 : 346.03 ms/batch | loss  4.00
Step  1000 : 339.11 ms/batch | loss  3.90
Step  1050 : 347.14 ms/batch | loss  4.34
Step  1100 : 346.12 ms/batch | loss  4.13
Step  1150 : 339.63 ms/batch | loss  4.06
Step  1200 : 342.82 ms/batch | los